In [1]:

import math, re, os


import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
from sklearn import metrics
from sklearn.model_selection import train_test_split
print("done")

done


In [2]:

try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.

    
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)
print("done")

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2023-11-22 00:47:16.206714: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:16.206840: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:16.206937: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:16.207012: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:16.207075: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:16.207303: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:16.207395: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8
done


In [3]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# Configuration
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
print(BATCH_SIZE)
EPOCHS = 468
print("done")

128
done


get_gcs_path is not required on TPU VMs which can directly use Kaggle datasets, using path: /kaggle/input/alaska2-image-steganalysis


In [4]:
def append_path(pre):
    return np.vectorize(lambda file: os.path.join(GCS_DS_PATH,pre,file))

In [5]:
sub = pd.read_csv('/kaggle/input/alaska2-image-steganalysis/sample_submission.csv')
train_filenames=np.array(os.listdir('/kaggle/input/alaska2-image-steganalysis/Cover/'))

In [6]:
np.random.seed(0)
positives = train_filenames.copy()
negatives = train_filenames.copy()

np.random.shuffle(positives)
np.random.shuffle(negatives)

jmipod=append_path('JMiPOD')(positives[:10000])
juniward=append_path('JUNIWARD')(positives[10000:])
uerd=append_path('UERD')(positives[20000:30000])

pos_paths=np.concatenate([jmipod, juniward,uerd])

In [7]:
np.random.seed(0)
positives = train_filenames.copy()
negatives = train_filenames.copy()

np.random.shuffle(positives)
np.random.shuffle(negatives)

jmipod=append_path('JMiPOD')(positives[:10000])
juniward=append_path('JUNIWARD')(positives[10000:])
uerd=append_path('UERD')(positives[20000:30000])

pos_paths=np.concatenate([jmipod, juniward,uerd])

In [8]:
test_paths=append_path('Test')(sub.Id.values)
neg_paths =append_path('Cover')(negatives[:30000])

In [9]:
train_paths=np.concatenate([pos_paths,neg_paths])
train_labels= np.array([1] * len(pos_paths)+[0] * len(neg_paths))
print("train_labels done")

train_labels done


In [10]:
train_paths, valid_paths, train_labels, valid_labels=train_test_split(train_paths,train_labels,test_size=0.2,random_state=42)
print("done")

done


In [11]:
def decode_image(filename, label=None, image_size=(512,512)):
    bits=tf.io.read_file(filename)
    image=tf.image.decode_jpeg(bits,channels=3)
    image=tf.cast(image,tf.float32) / 255.0 #image to tf.float32 data type
    image=tf.image.resize(image,image_size)
    
    if label is None:
        return image
    else:
        return image,label

def data_augment(image,label=None):
    image=tf.image.random_flip_left_right(image)
    image=tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image,label

In [12]:
train_dataset = (tf.data.Dataset
                 .from_tensor_slices((train_paths,train_labels))
                 .map(decode_image, num_parallel_calls=AUTO)
                 .cache()
                 .repeat()
                 .shuffle(1024)
                 .batch(BATCH_SIZE)
                 .prefetch(AUTO)
                )
valid_dataset= (tf.data.Dataset
                .from_tensor_slices((valid_paths,valid_labels))
                .map(decode_image, num_parallel_calls=AUTO)
                .batch(BATCH_SIZE)
                .prefetch(AUTO)
               )
test_dataset= (tf.data.Dataset
               .from_tensor_slices(test_paths)
               .map(decode_image, num_parallel_calls=AUTO)
               .batch(BATCH_SIZE)
              )

In [13]:
def build_lrfn(lr_start=0.0001,lr_max=0.000075,
              lr_min=0.000001, lr_rampup_epochs=20,
              lr_sustain_epochs=0,lr_exp_decay=0.8):
    lr_max = lr_max * strategy.num_replicas_in_sync
    
    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max- lr_start)/lr_rampup * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay ** (epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
        
    return lrfn

In [14]:
from tensorflow.keras.applications.resnet50 import ResNet50
print("imported resnet")

imported resnet


In [15]:
with strategy.scope():
    model = tf.keras.Sequential([
        ResNet50(
        input_shape=(512,512,3),
        weights='imagenet',
        include_top=False),
        L.GlobalAveragePooling2D(),
        L.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['accuracy']
    )
    model.summary()

2023-11-22 00:47:24.269881: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:24.269997: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


94765736/94765736 [==============================] - 0s 0us/step


2023-11-22 00:47:34.206165: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:34.206756: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:34.207272: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:34.207708: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:34.208359: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:34.209244: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:47:34.209958: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 16, 16, 2048)      23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23589761 (89.99 MB)
Trainable params: 23536641 (89.79 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [16]:
STEPS_PER_EPOCH=train_labels.shape[0] // BATCH_SIZE

history= model.fit(
train_dataset,
epochs=EPOCHS,
steps_per_epoch=STEPS_PER_EPOCH,
validation_data=valid_dataset)

Epoch 1/468


2023-11-22 00:48:07.706920: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.
2023-11-22 00:48:11.072644: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:48:11.073884: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:48:11.074027: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:48:11.080608: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:48:11.082533: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-22 00:48:11.086203: E ./tensorflow/compiler/xla/stream_executor/st

  2/718 [..............................] - ETA: 9:13 - loss: 0.9745 - accuracy: 0.6602    

2023-11-22 00:48:48.186575: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


  6/718 [..............................] - ETA: 8:29 - loss: 1.0713 - accuracy: 0.6497

2023-11-22 00:48:50.994488: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


718/718 [==============================] - ETA: 0s - loss: 0.5871 - accuracy: 0.7349

2023-11-22 00:56:48.227722: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


718/718 [==============================] - 638s 805ms/step - loss: 0.5871 - accuracy: 0.7349 - val_loss: 0.5740 - val_accuracy: 0.7451
Epoch 2/468
718/718 [==============================] - 535s 746ms/step - loss: 0.5777 - accuracy: 0.7374 - val_loss: 0.5883 - val_accuracy: 0.7389
Epoch 3/468
718/718 [==============================] - 537s 748ms/step - loss: 0.5768 - accuracy: 0.7377 - val_loss: 0.5805 - val_accuracy: 0.7341
Epoch 4/468
718/718 [==============================] - 537s 748ms/step - loss: 0.5772 - accuracy: 0.7376 - val_loss: 0.5717 - val_accuracy: 0.7451
Epoch 5/468
718/718 [==============================] - 534s 743ms/step - loss: 0.5770 - accuracy: 0.7376 - val_loss: 0.5773 - val_accuracy: 0.7451
Epoch 6/468
718/718 [==============================] - 538s 750ms/step - loss: 0.5770 - accuracy: 0.7377 - val_loss: 0.5678 - val_accuracy: 0.7451
Epoch 7/468
301/718 [===========>..................] - ETA: 4:20 - loss: 0.5770 - accuracy: 0.7378

KeyboardInterrupt: 